# Load database

In [6]:
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import os
from dotenv import load_dotenv

In [7]:
# Carregar variáveis de ambiente do arquivo .env
load_dotenv()

# Obter caminhos do arquivo csv e do diretório de extração das variáveis de ambiente
propostas_credito_path = os.getenv('PROPOSTAS_CREDITO_CSV')

In [8]:
# Carregar o CSV em um DataFrame Pandas
df_propostas_credito = pd.read_csv(propostas_credito_path)

**Wrangling**

Vamos realizar a manipulação dos dados para o dashboard, priorizando a obtenção de uma granularidade consistente e aprimoramento da qualidade da base de dados.

In [9]:
df_propostas_credito.head()

,cod_proposta,cod_cliente,cod_colaborador,data_entrada_proposta,taxa_juros_mensal,valor_proposta,valor_financiamento,valor_entrada,valor_prestacao,quantidade_parcelas,carencia,status_proposta
0,116,338,1,2014-05-30 18:43:12 UTC,0.0194,36199.950355,50032.03,13832.079645,1045.219825,58,0,Enviada
1,715,45,1,2021-06-17 13:09:32 UTC,0.0131,12897.538285,19848.55,6951.011715,232.124685,100,1,Enviada
2,755,494,1,2021-08-10 04:24:02 UTC,0.0203,122236.312551,187154.36,64918.047449,3673.518951,56,0,Enviada
3,953,381,1,2011-02-23 07:57:58 UTC,0.0167,47784.649757,59530.61,11745.960243,955.041560,109,3,Enviada
4,1046,176,1,2010-11-15 16:03:28 UTC,0.0211,178393.038243,214294.19,35901.151757,10219.659415,22,4,Enviada


In [10]:
df_propostas_credito.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   cod_proposta           2000 non-null   int64  
 1   cod_cliente            2000 non-null   int64  
 2   cod_colaborador        2000 non-null   int64  
 3   data_entrada_proposta  2000 non-null   object 
 4   taxa_juros_mensal      2000 non-null   float64
 5   valor_proposta         2000 non-null   float64
 6   valor_financiamento    2000 non-null   float64
 7   valor_entrada          2000 non-null   float64
 8   valor_prestacao        2000 non-null   float64
 9   quantidade_parcelas    2000 non-null   int64  
 10  carencia               2000 non-null   int64  
 11  status_proposta        2000 non-null   object 
dtypes: float64(5), int64(5), object(2)
memory usage: 187.6+ KB


Vamos fazer um type casting da coluna data_entrada_proposta para o formato de 'datetime64[ns]'

A coluna data_entrada_proposta está formatada com o sufixo "UTC". Para lidar com isso, usaremos a função pd.to_datetime() do pandas, que oferece mais flexibilidade para o parsing de strings de data.

In [11]:
# Converter corretamente as strings de data com o sufixo "UTC" para objetos datetime do pandas
df_propostas_credito['data_entrada_proposta'] = pd.to_datetime(df_propostas_credito['data_entrada_proposta'], utc=True)

# remover a informação de fuso horário, convertendo as datas para o formato local naive datetime
df_propostas_credito['data_entrada_proposta'] = df_propostas_credito['data_entrada_proposta'].dt.tz_localize(None)

In [12]:
df_propostas_credito['data_entrada_proposta'] = df_propostas_credito['data_entrada_proposta'].astype('datetime64[ns]')

Enriquecimento dos dados com novas colunas.

Chaves temporais:

In [14]:
df_propostas_credito['data_entrada_proposta_month'] = df_propostas_credito['data_entrada_proposta'].apply(lambda date: date.strftime('%Y-%m'))
df_propostas_credito['data_entrada_proposta_year']  = df_propostas_credito['data_entrada_proposta'].apply(lambda date: date.strftime('%Y'))

Com os dados manipulados, procederemos à sua persistência em disco, realizaremos o download e, posteriormente, o carregaremos no Google Data Studio.

In [15]:
# Obter caminhos do diretório gold das variáveis de ambiente
gold_path = os.getenv('GOLD_PATH')

In [16]:
df_propostas_credito.to_csv(f'{gold_path}/propostas_credito.csv', sep=',', index=False)